# 🚀 Task 9: Setup YOLOv11 with Ultralytics

## 🎯 Objective
Set up YOLOv11 using the Ultralytics library and prepare for training on our waste classification dataset.

---

## 📚 About Ultralytics

Ultralytics provides:
- Easy YOLO model loading and training
- Pre-trained weights for transfer learning
- Built-in data augmentation
- Training callbacks and logging

### ML Rules Applied:
- **Rule #4**: Keep the first model simple
- **Rule #14**: Starting with an interpretable model
- **Rule #38**: Test the model on new data

In [ ]:
# Install ultralytics if not already installed
# !pip install ultralytics

import os
import sys
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import yaml

# Ultralytics
from ultralytics import YOLO

# Project paths
PROJECT_ROOT = Path(r"D:\het\SELF\RP\YOLO-V11-PRO")
DATA_DIR = PROJECT_ROOT / "data" / "processed"
MODELS_DIR = PROJECT_ROOT / "models"
MODELS_DIR.mkdir(exist_ok=True)

print("✅ Libraries imported!")
print(f"📁 Project: {PROJECT_ROOT}")
print(f"📁 Data: {DATA_DIR}")

---

# Part 1: Verify Dataset

First, let's verify our dataset is ready for training.

In [ ]:
# Check dataset structure
def verify_dataset():
    """Verify dataset is properly formatted for YOLO."""
    
    dirs_to_check = [
        DATA_DIR / "images" / "train",
        DATA_DIR / "images" / "val",
        DATA_DIR / "labels" / "train",
        DATA_DIR / "labels" / "val",
    ]
    
    print("📁 Dataset Structure Check:")
    print("="*50)
    
    all_ok = True
    for d in dirs_to_check:
        if d.exists():
            count = len(list(d.glob("*")))
            print(f"   ✅ {d.name:15} : {count:5} files")
        else:
            print(f"   ❌ {d.name:15} : MISSING")
            all_ok = False
    
    # Check dataset.yaml
    yaml_path = DATA_DIR / "dataset.yaml"
    if yaml_path.exists():
        print(f"   ✅ {'dataset.yaml':15} : EXISTS")
        with open(yaml_path, 'r') as f:
            config = yaml.safe_load(f)
            print(f"\n📄 Dataset Config:")
            print(f"   Path: {config.get('path', 'N/A')}")
            print(f"   Classes: {config.get('names', 'N/A')}")
    else:
        print(f"   ❌ {'dataset.yaml':15} : MISSING")
        all_ok = False
    
    return all_ok

dataset_ready = verify_dataset()

---

# Part 2: Load YOLOv11 Model

## Model Variants

| Model | Size | mAP | Speed |
|-------|------|-----|-------|
| yolo11n | Nano | 39.5 | Fastest |
| yolo11s | Small | 47.0 | Fast |
| yolo11m | Medium | 51.5 | Moderate |
| yolo11l | Large | 53.4 | Slow |
| yolo11x | XLarge | 54.7 | Slowest |

We'll use **yolo11n** (nano) for faster training.

In [ ]:
# ============================================================
# LOAD YOLOV11 MODEL
# ============================================================

# Load pretrained model (nano for speed)
model = YOLO('yolo11n.pt')  # Downloads automatically if not present

print("\n✅ YOLOv11 Nano Model Loaded!")
print(f"\n📊 Model Info:")
print(f"   Type: {type(model).__name__}")
print(f"   Task: Detection")

In [ ]:
# Model architecture summary
print("\n📐 Model Architecture Summary:")
print("="*50)
model.info()

---

# Part 3: Training Configuration

Key hyperparameters for training:

In [ ]:
# ============================================================
# TRAINING CONFIGURATION
# ============================================================

train_config = {
    # Dataset
    'data': str(DATA_DIR / 'dataset.yaml'),
    
    # Training parameters
    'epochs': 50,           # Number of training epochs
    'batch': 16,            # Batch size (adjust based on GPU memory)
    'imgsz': 640,           # Input image size
    
    # Optimizer
    'optimizer': 'AdamW',   # Optimizer choice
    'lr0': 0.01,            # Initial learning rate
    'lrf': 0.01,            # Final learning rate factor
    
    # Augmentation
    'augment': True,        # Enable augmentation
    'hsv_h': 0.015,         # Hue augmentation
    'hsv_s': 0.7,           # Saturation augmentation
    'hsv_v': 0.4,           # Value augmentation
    'flipud': 0.5,          # Vertical flip probability
    'fliplr': 0.5,          # Horizontal flip probability
    'mosaic': 1.0,          # Mosaic augmentation
    
    # Regularization
    'weight_decay': 0.0005, # L2 regularization
    
    # Output
    'project': str(MODELS_DIR),
    'name': 'waste_yolo11n',
    'exist_ok': True,
    
    # Performance
    'device': 0,            # GPU device (0 for first GPU, 'cpu' for CPU)
    'workers': 4,           # Number of data loading workers
    
    # Logging
    'verbose': True,
    'plots': True,          # Generate training plots
}

print("✅ Training Configuration:")
print("="*50)
for key, value in train_config.items():
    print(f"   {key:15}: {value}")

In [ ]:
# Save configuration
config_path = PROJECT_ROOT / 'configs' / 'train_config.yaml'
config_path.parent.mkdir(exist_ok=True)

with open(config_path, 'w') as f:
    yaml.dump(train_config, f, default_flow_style=False)

print(f"✅ Configuration saved: {config_path}")

---

# Part 4: Quick Validation Test

Test model inference before training:

In [ ]:
# Test inference on sample image
from PIL import Image

# Get sample image
sample_images = list((DATA_DIR / 'images' / 'train').glob('*.jpg'))[:1]

if sample_images:
    sample_path = sample_images[0]
    print(f"\n🖼️ Testing inference on: {sample_path.name}")
    
    # Run inference (pre-trained on COCO)
    results = model.predict(source=str(sample_path), conf=0.25, save=False)
    
    print(f"\n✅ Inference completed!")
    print(f"   Detections: {len(results[0].boxes)}")
    
    # Display
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    
    # Original
    img = Image.open(sample_path)
    axes[0].imshow(img)
    axes[0].set_title('Original Image')
    axes[0].axis('off')
    
    # With detections
    result_img = results[0].plot()
    axes[1].imshow(result_img)
    axes[1].set_title('Pre-trained Model Detections (COCO)')
    axes[1].axis('off')
    
    plt.tight_layout()
    plt.savefig(PROJECT_ROOT / 'docs' / 'assets' / 'pretrained_inference.png', dpi=150)
    plt.show()
else:
    print("❌ No sample images found")

---

# Part 5: Training Script Template

Ready-to-run training code:

In [ ]:
# ============================================================
# TRAINING SCRIPT (Run this to train)
# ============================================================

def train_model(config):
    """
    Train YOLOv11 on waste classification dataset.
    
    Args:
        config: Training configuration dictionary
    
    Returns:
        Training results
    """
    print("\n" + "="*60)
    print("🚀 STARTING YOLOV11 TRAINING")
    print("="*60)
    
    # Load model
    model = YOLO('yolo11n.pt')
    
    # Train
    results = model.train(**config)
    
    print("\n" + "="*60)
    print("✅ TRAINING COMPLETE!")
    print("="*60)
    
    return results

# Uncomment to train:
# results = train_model(train_config)

print("\n📝 Training function ready!")
print("   Uncomment the last line to start training.")

## 📝 Summary

### Setup Complete:
1. ✅ Dataset verified
2. ✅ YOLOv11 nano model loaded
3. ✅ Training configuration prepared
4. ✅ Inference tested
5. ✅ Training script ready

### Key Files:
- `data/processed/dataset.yaml` - Dataset config
- `configs/train_config.yaml` - Training config
- `models/waste_yolo11n/` - Training outputs

### Next: Task 10 - Model Training!

In [ ]:
print("\n" + "="*60)
print("✅ TASK 9 COMPLETE: YOLOv11 Setup")
print("="*60)
print("\n📋 What was accomplished:")
print("   ✓ Dataset structure verified")
print("   ✓ YOLOv11 model loaded")
print("   ✓ Training config created")
print("   ✓ Pre-training inference tested")
print("   ✓ Training script prepared")
print("\n➡️ Ready for Task 10: Model Configuration")